In [6]:
%pwd

'c:\\Users\\pc\\Desktop\\Medicalchatbot\\Medicalchatbot\\research'

In [11]:
import os
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\pc\\Desktop\\Medicalchatbot\\Medicalchatbot'

In [12]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter 

In [13]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [14]:
extracted_data = load_pdf_file(data='Data/')

In [15]:
#Split the Data into Text Chunks

def text_split(extracted_data):

    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

    text_chunks=text_splitter.split_documents(extracted_data)

    return text_chunks

In [16]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [17]:
from langchain.embeddings import HuggingFaceEmbeddings

In [18]:
#Download the Embeddings from Hugging Face

def download_hugging_face_embeddings():
	embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
	return embeddings 


In [19]:
embeddings = download_hugging_face_embeddings()

C:\Users\pc\AppData\Local\Temp\ipykernel_12016\3877738725.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [20]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [83]:
from dotenv import load_dotenv
load_dotenv()


True

In [84]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [22]:
from langchain_pinecone import PineconeVectorStore

index_name = 'medicalchatbot'  # Define the index_name variable

docsearch = PineconeVectorStore.from_documents(
documents=text_chunks,
index_name=index_name,
embedding=embeddings,
)

In [23]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore

# Embed each chunk and upsert the embeddings into your Pinecone index.

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [24]:
docsearch

In [62]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [63]:
retrieved_docs = retriever.invoke("What is acne?")

In [64]:
retrieved_docs

[Document(id='fe167ca3-4e9e-494f-b9a1-3180730404a8', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='42b62f28-c47d-4fc8-8d8f-6acc0bfdac56', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='ddb251ba-3363-4bf6-8595-3b805c09ae43', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\Gale Encyclopedia of Medicine. Vol. 1. 2nd Edition ( PDFDrive ).pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26')]

In [94]:

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro', temperature=0.8, max_tokens=500)

In [87]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [88]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [98]:
response = rag_chain.invoke({"input": "What is acne?"})
print(response["answer"])

This content does not contain the definition of acne.  Therefore, I am unable to answer your question.  Please provide more context.


In [97]:
response = rag_chain.invoke({"input": "What is jaundice?"})
print(response["answer"])

Jaundice is the first symptom in 90% of patients with bile duct tumors.  It occurs when a bile duct tumor obstructs the normal flow of bile, causing bilirubin to build up. This buildup results in a yellowing of the skin and eyes.
